<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению. Сессия № 2
</center>
Автор материала: программист-исследователь Mail.ru Group, старший преподаватель Факультета Компьютерных Наук ВШЭ Юрий Кашницкий. Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Занятие 8. Разреженные данные, онлайн-обучение</center>

## <center>Часть 1. Категориальные признаки</center>

### <center>One-hot encoding</center>

Подавляющее большинство методов классификации и регрессии сформулированы в терминах евклидовых или метрических пространств, то есть подразумевают представление данных в виде вещественных векторов одинаковой размерности. В реальных данных, однако, не так редки категориальные атрибуты, принимающие дискретные значения, такие как да/нет или  январь/февраль/.../декабрь. В ходе этого занятия мы научимся работать с такими данными, в частности с помощью линейных моделей.

In [ ]:
import pandas as pd
import sklearn.feature_extraction
import sklearn.linear_model
import sklearn.metrics
import sklearn.model_selection
import sklearn.preprocessing

%matplotlib inline

from pprint import pformat

import matplotlib.pyplot as plt

plt.style.use("ggplot")
import warnings

warnings.filterwarnings("ignore")

В этой части занятия мы рассмотрим выборку bank:

In [ ]:
df = pd.read_csv("../../data/bank_train.csv")
labels = pd.read_csv("../../data/bank_train_target.csv", header=None)

df.head()

Нетрудно заметить, что достаточно много признаков в этом наборе данных не представлены числами. В таком виде данные еще нам не подходят - мы не сможем применять подавляющее большинство доступных нам методов.

Чтобы найти решение, давайте рассмотрим атрибут education:

In [ ]:
df["education"].value_counts().plot.barh();

Естественным решением такой проблемы было бы однозначное отображение каждого значения в уникальное число. К примеру, мы могли бы преобразовать university.degree в 0, а basic.9y в 1. Эту простую операцию приходится делать часто, поэтому в модуле preprocessing библиотеки sklearn именно для этой задачи реализован класс LabelEncoder:

In [ ]:
label_encoder = sklearn.preprocessing.LabelEncoder()

Метод fit этого класса находит все уникальные значения и строит таблицу для соответствия каждой категории некоторому числу, а метод transform непосредственно преобразует значения в числа. После fit у label_encoder будет доступно поле classes, содержащее все уникальные значения. Пронумеруем их, чтобы убедиться, что преобразование выполнено верно. 

In [ ]:
mapped_education = pd.Series(label_encoder.fit_transform(df["education"]))
mapped_education.value_counts().plot.barh()
print(dict(enumerate(label_encoder.classes_)))

Что произойдет, если у нас появятся данные с другими категориями?

In [ ]:
try:
    label_encoder.transform(df["education"].replace("high.school", "high_school"))
except Exception as e:
    print("Error:", e)

Таким образом, при использовании этого подхода мы всегда должны быть уверены, что признак не может принимать неизвестных ранее значений. К этой проблеме мы вернемся чуть позже, а сейчас заменим весь столбец education на преобразованный:

In [ ]:
df["education"] = mapped_education
df.head()

Продолжим преобразование для всех столбцов, имеющих тип object - именно этот тип задается в pandas для таких данных.

In [ ]:
categorical_columns = df.columns[df.dtypes == "object"].union(["education"])
for column in categorical_columns:
    df[column] = label_encoder.fit_transform(df[column])
df.head()

Основная проблема такого представления заключается в том, что числовой код создал евклидово представление для данных.

К примеру, нами неявным образом была введена алгебра над значениями работы - мы можем вычесть работу клиента 1 из работы клиента 2:

In [ ]:
df.loc[1].job - df.loc[2].job

Конечно же, эта операция не имеет никакого смысла. Но именно на этом основаны метрики близости объектов, что делает бессмысленным применение метода ближайшего соседа на данных в таком виде. Аналогичным образом, никакого смысла не будет иметь применение линейных моделей. Убедимся в этом:

In [ ]:
def logistic_regression_accuracy_on(dataframe, labels):
    features = dataframe.as_matrix()
    (
        train_features,
        test_features,
        train_labels,
        test_labels,
    ) = sklearn.model_selection.train_test_split(features, labels)

    lr = sklearn.linear_model.LogisticRegression()
    lr.fit(train_features, train_labels)
    return sklearn.metrics.classification_report(test_labels, lr.predict(test_features))


print(logistic_regression_accuracy_on(df[categorical_columns], labels))

Для того, чтобы мы смогли применять линейные модели на таких данных нам необходим другой метод, который называется one-hot encoding.

Предположим, что некоторый признак может принимать 10 разных значений. В этом случае one hot encoding подразумевает создание 10 признаков, все из которых равны нулю *за исключением одного*. На позицию, соответствующую численному значению признака мы помещаем 1:

In [ ]:
one_hot_example = pd.DataFrame([{i: 0 for i in range(10)}])
one_hot_example.loc[0, 6] = 1
one_hot_example

Эта техника реализована в sklearn в классе OneHotEncoder. По умолчанию OneHotEncoder преобразует данные в разреженную матрицу, чтобы не расходовать память на хранение многочисленных нулей. Однако в этом примере размер данных не является для нас проблемой, поэтому мы будем использовать "плотное" представление.

In [ ]:
onehot_encoder = sklearn.preprocessing.OneHotEncoder(sparse=False)

Преобразуем категориальные столбцы с помощью OneHotEncoder:

In [ ]:
encoded_categorical_columns = pd.DataFrame(
    onehot_encoder.fit_transform(df[categorical_columns])
)
encoded_categorical_columns.head()

Мы получили 53 столбца - именно столько различных уникальных значений могут принимать категориальные столбцы исходной выборки. Преобразованные с помощью one-hot encoding данные начинают обретать смысл для линейной модели:

In [ ]:
print(logistic_regression_accuracy_on(encoded_categorical_columns, labels))

### <center>Hashing trick</center>

Реальные данные могут оказаться гораздо более динамичными и мы не всегда можем рассчитывать, что категориальные признаки не будут принимать новых значений. Все это сильно затрудняет использование уже обученных моделей на новых данных. Кроме того, LabelEncoder подразумевает предварительный анализ всей выборки и хранение построенных отображений в памяти, что затрудняет работу в режиме больших данных.

Для решения этих проблем существует более простой подход к векторизации категориальных признаков, основанный на хэшировании, известный как hashing trick. 

Хэш-функции могут помочь нам в задаче поиска уникальных кодов для различных значений атрибутов, к примеру:

In [ ]:
for s in ("university.degree", "high.school", "illiterate"):
    print(s, "->", hash(s))

Отрицательные и настолько большие по модулю значения нам не подойдут. Ограничим область значений хэш-функции:

In [ ]:
hash_space = 25
for s in ("university.degree", "high.school", "illiterate"):
    print(s, "->", hash(s) % hash_space)

Представим, что у нас в выборке есть холостой студент, которому позвонили в понедельник, тогда его вектор признаков будет сформирован аналогично one-hot encoding, но в **едином пространстве фиксированного размера для всех признаков**:

In [ ]:
hashing_example = pd.DataFrame([{i: 0.0 for i in range(hash_space)}])
for s in ("job=student", "marital=single", "day_of_week=mon"):
    print(s, "->", hash(s) % hash_space)
    hashing_example.loc[0, hash(s) % hash_space] = 1
hashing_example

Стоит обратить внимание, что в этом примере хэшировались не только значения атрибутов, а **пары название атрибута + значение атрибута**. Это необходимо, чтобы разделить одинаковые значения разных атрибутов между собой, к примеру:

In [ ]:
assert hash("no") == hash("no")
assert hash("housing=no") != hash("loan=no")

Может ли произойти коллизия хэш-функции, то есть совпадение кодов для двух разных значений? Нетрудно доказать, что при достаточном размере пространства хэширования это происходит редко, но даже в тех случаях, когда это происходит, это не будет приводить к существенному ухудшению качества классификации или регрессии.

В следующей части занятия мы будем применять подход на основе хэширования для работы с категориальными данными.